In [27]:
import pandas as pd
from sklearn.metrics import confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split,RandomizedSearchCV
import xgboost as xgb

In [28]:
df=pd.read_csv("../data/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [29]:
X=df.drop("Class",axis=1)
y=df["Class"]

In [30]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
print("Original dataset fraud percentage:", y.value_counts(normalize=True)[1])
print("Training set fraud percentage:", y_train.value_counts(normalize=True)[1])
print("Test set fraud percentage:", y_test.value_counts(normalize=True)[1])

Original dataset fraud percentage: 0.001727485630620034
Training set fraud percentage: 0.001729245759178389
Test set fraud percentage: 0.0017204452090867595


In [31]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)
print("\nShape of y_train before SMOTE:", y_train.shape)
print("Shape of y_train after SMOTE:", y_train_resampled.shape)
print("\nValue counts of y_train after SMOTE:\n", y_train_resampled.value_counts())


Shape of y_train before SMOTE: (227845,)
Shape of y_train after SMOTE: (454902,)

Value counts of y_train after SMOTE:
 Class
0    227451
1    227451
Name: count, dtype: int64


In [32]:
#train the model
model=xgb.XGBClassifier(objective="binary:logistic",
                        eval_metric='logloss')
print("Training XGBoost model")
model.fit(X_train_resampled,y_train_resampled)
print("Training complete")

Training XGBoost model
Training complete


In [33]:
#prediction and evaluation
y_pred=model.predict(x_test)
print("\n--- XGBoost Model Evaluation ---")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


--- XGBoost Model Evaluation ---
Confusion Matrix:
[[56842    22]
 [   15    83]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.79      0.85      0.82        98

    accuracy                           1.00     56962
   macro avg       0.90      0.92      0.91     56962
weighted avg       1.00      1.00      1.00     56962



In [47]:
#now hypertune and compare
params={
    'max_depth':[3,4,5,6,7],
    'learning_rate':[0.01, 0.05, 0.1, 0.2],
    'n_estimators':[100, 200, 300, 500],
    'subsample':[0.7, 0.8, 0.9, 1.0],
    'colsample_bytree':[0.7, 0.8, 0.9, 1.0]
}

random_search=RandomizedSearchCV(
    estimator=model,
    param_distributions=params,
    n_iter=25,
    scoring='f1',
    n_jobs=-1,
    cv=3,
    verbose=2,
    random_state=42
)

print("Starting hyperparameter tuning...")
random_search.fit(X_train_resampled, y_train_resampled)
print("Tuning complete.")

# Getting the Best Parameters
print("\nBest parameters found:")
print(random_search.best_params_)

Starting hyperparameter tuning...
Fitting 3 folds for each of 25 candidates, totalling 75 fits
Tuning complete.

Best parameters found:
{'subsample': 1.0, 'n_estimators': 500, 'max_depth': 7, 'learning_rate': 0.1, 'colsample_bytree': 0.7}


In [48]:
#now prdicting using the best model we found during hyperparameter tuning
best_model=random_search.best_estimator_
y_best_pred=best_model.predict(x_test)
print("\n--- XGBoost Model Evaluation after tuning---")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_best_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_best_pred))


--- XGBoost Model Evaluation after tuning---
Confusion Matrix:
[[56847    17]
 [   15    83]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.85      0.84        98

    accuracy                           1.00     56962
   macro avg       0.91      0.92      0.92     56962
weighted avg       1.00      1.00      1.00     56962

